In [ ]:
from pyspark.sql.session import SparkSession
import pyspark.sql.functions as f

In [ ]:
spark = SparkSession.builder.getOrCreate()

In [ ]:
emp = (
    spark.read.csv("EMPRESAS_CNPJ.csv", sep=';')
    .toDF(
        'cnpj_basico',
        'razao_social',
        'natureza_juridica',
        'qualificacao_responsavel',
        'capital_social',
        'porte',
        'ente_responsavel')
)

est = (
    spark.read.csv("ESTABELE_CNPJ.csv", sep=';')
    .toDF(
        'cnpj_basico',
        'cnpj_ordem',
        'cnpj_dv',
        'id_matriz',
        'nome_fantasia',
        'situacao',
        'situacao_data',
        'situacao_motivo',
        'cidade_exterior',
        "pais_codigo",
        'inicio_data',
        'cnae_principal',
        'cnae_secundario',
        'logradouro_tipo',
        'logradouro',
        'lograoduro_numero',
        'logradouro_complemento',
        'bairro',
        'cep',
        'uf',
        'municipio',
        'ddd1',
        'telefone1',
        'ddd2',
        'telefone2',
        'ddd_fax',
        'fax',
        'email',
        'situacao_especial',
        'situacao_especial_data')
)

In [ ]:
emp = (
    emp
    .select(
        'cnpj_basico',
        'natureza_juridica',
        'qualificacao_responsavel',
        'capital_social',
        'porte',
    )
)

est = (
    est
    .select(
        'cnpj_basico',
        'cnpj_ordem',
        'cnpj_dv',
        'id_matriz',
        'situacao',
        'situacao_data',
        'situacao_motivo',
        'inicio_data',
        'cnae_principal',
        'cep',
        'uf',
    )
)

In [ ]:
est = est.filter(f.col('uf') == 'MG')

In [ ]:
est = (
    est
    .join(emp, on='cnpj_basico', how='inner')
)

### Formatando as datas

In [ ]:
est = (
    est
    .withColumn('situacao_data', f.to_date(f.col('situacao_data').cast('string'), "yyyyMMdd"))
    .withColumn('inicio_data', f.to_date(f.col('inicio_data').cast('string'), "yyyyMMdd"))
)

In [ ]:
est = (
    est
    .withColumn('cnpj', f.concat_ws("",
                f.col('cnpj_basico'), f.col('cnpj_ordem'), f.col('cnpj_dv')
                ))
    .drop('cnpj_basico', 'cnpj_dv', 'cnpj_ordem')
)

In [ ]:
est.groupBy('natureza_juridica').count().show(500)